In [2]:
import numpy as np
import time

class Particle:
    def __init__(self, n_cities):
        # Inisialisasi partikel dengan posisi acak (urutan kota) dan kecepatan nol
        self.position = np.random.permutation(n_cities)
        self.velocity = np.zeros(n_cities)
        self.best_position = self.position.copy()
        self.best_score = float('inf')

class PSO:
    def __init__(self, distance_matrix, n_particles, iterations, w=0.5, c1=1, c2=2):
        # Inisialisasi parameter PSO
        self.distance_matrix = distance_matrix
        self.n_cities = len(distance_matrix)
        self.n_particles = n_particles
        self.iterations = iterations
        self.w = w  # Bobot inersia
        self.c1 = c1  # Koefisien kognitif
        self.c2 = c2  # Koefisien sosial
        self.global_best_position = None
        self.global_best_score = float('inf')
        self.particles = [Particle(self.n_cities) for _ in range(n_particles)]

    def calculate_total_distance(self, route):
        # Menghitung total jarak rute
        return sum(self.distance_matrix[route[i]][route[i+1]] for i in range(self.n_cities-1)) + self.distance_matrix[route[-1]][route[0]]

    def update_velocity(self, particle):
        # Update kecepatan partikel
        r1, r2 = np.random.rand(2)
        particle.velocity = (self.w * particle.velocity + 
                             self.c1 * r1 * (particle.best_position - particle.position) + 
                             self.c2 * r2 * (self.global_best_position - particle.position))

    def update_position(self, particle):
        # Update posisi partikel
        new_position = particle.position + particle.velocity
        particle.position = np.argsort(new_position)

    def run(self):
        # Jalankan algoritma PSO
        for _ in range(self.iterations):
            for particle in self.particles:
                score = self.calculate_total_distance(particle.position)
                # Update personal best
                if score < particle.best_score:
                    particle.best_score = score
                    particle.best_position = particle.position.copy()
                # Update global best
                if score < self.global_best_score:
                    self.global_best_score = score
                    self.global_best_position = particle.position.copy()

            # Update kecepatan dan posisi semua partikel
            for particle in self.particles:
                self.update_velocity(particle)
                self.update_position(particle)

        return self.global_best_position, self.global_best_score

def main():
    # Load matrix jarak
    distance_matrix = np.loadtxt(open("dataset.txt", 'rb'), delimiter=' ')

    # Definisikan cluster
    clusters = [
        [0, 2, 13, 9, 14, 10, 11, 19],
        [0, 32, 34, 40, 60, 63, 67, 68, 69],
        [0, 20, 29, 37, 41, 44, 45, 46, 47, 49],
        [0, 16, 21, 22, 23, 24, 25, 26, 28, 30],
        [0, 1, 52, 53, 54, 55, 56, 57, 58, 59, 65],
        [0, 31, 33, 35, 36, 39, 42, 43, 48, 50, 51],
        [0, 3, 4, 5, 6, 7, 8, 12, 61],
        [0, 15, 17, 18, 27, 38, 62, 64, 66, 70],
    ]

    N_PARTICLES = 50
    ITERATIONS = 100

    total_shortest_distance = 0
    total_runtime = 0

    # Jalankan PSO untuk setiap cluster
    for i, cluster in enumerate(clusters):
        selected_distance_matrix = distance_matrix[cluster][:, cluster]
        
        start_time = time.time()
        pso = PSO(selected_distance_matrix, N_PARTICLES, ITERATIONS)
        best_route, shortest_distance = pso.run()
        end_time = time.time()
        runtime = end_time - start_time
        
        total_runtime += runtime
        total_shortest_distance += shortest_distance

        final_route = [cluster[index] for index in best_route]
        
        # Cetak hasil untuk setiap cluster
        print(f"Cluster {i + 1}:")
        print("Route:", final_route)
        print("Shortest distance:", shortest_distance)
        print(f"Runtime: {runtime:.2f} seconds")
        print()

    # Cetak hasil total
    print("Total Shortest Distance for all clusters:", total_shortest_distance)
    print(f"Total Runtime: {total_runtime:.2f} seconds")

if __name__ == '__main__':
    main()

Cluster 1:
Route: [0, 19, 14, 9, 11, 13, 10, 2]
Shortest distance: 31.1
Runtime: 0.07 seconds

Cluster 2:
Route: [69, 63, 40, 34, 60, 32, 67, 0, 68]
Shortest distance: 25.1
Runtime: 0.07 seconds

Cluster 3:
Route: [20, 0, 49, 45, 44, 41, 46, 37, 47, 29]
Shortest distance: 39.9
Runtime: 0.07 seconds

Cluster 4:
Route: [30, 21, 24, 0, 26, 25, 23, 16, 28, 22]
Shortest distance: 41.29999999999999
Runtime: 0.07 seconds

Cluster 5:
Route: [52, 56, 59, 58, 55, 0, 54, 1, 65, 53, 57]
Shortest distance: 26.500000000000004
Runtime: 0.07 seconds

Cluster 6:
Route: [39, 50, 36, 35, 48, 0, 43, 31, 51, 33, 42]
Shortest distance: 39.7
Runtime: 0.07 seconds

Cluster 7:
Route: [4, 61, 0, 5, 7, 12, 6, 8, 3]
Shortest distance: 49.5
Runtime: 0.07 seconds

Cluster 8:
Route: [15, 27, 62, 70, 0, 66, 38, 64, 17, 18]
Shortest distance: 21.8
Runtime: 0.06 seconds

Total Shortest Distance for all clusters: 274.9
Total Runtime: 0.55 seconds
